In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/sskyisthelimit/DS_Internship.git

In [ ]:
%cd DS_Internship/CV/src/

In [ ]:
from utils import Sentinel2Downloader, load_torch_image, split_image_and_save
API_KEY = "/content/drive/MyDrive/Datasets/MATCHING/image-matching-key.json"

In [ ]:
CONSTRAINTS = {'NODATA_PIXEL_PERCENTAGE': 15.0,
               'CLOUDY_PIXEL_PERCENTAGE': 10.0, }

loader = Sentinel2Downloader(api_key=API_KEY)

loaded = loader.download(product_type="L2A",
                        tiles=["36TVS"],
                        start_date="2018-12-01",
                        end_date="2021-12-01",
                        output_dir="/content/sentinel2imagery",
                        bands=set(["TCI"]),
                        cores=4,
                        constraints=CONSTRAINTS)

In [ ]:
import os

images_num = 0
image_names_dict = {}

for dirname, _, filenames in os.walk('/content/sentinel2imagery'):
    for filename in filenames:
        if filename.endswith('_TCI_10m.jp2'):
            images_num += 1
            if dirname not in image_names_dict:
                image_names_dict[dirname] = list()
            image_names_dict[dirname].append(filename)

print(images_num)
print(len([*image_names_dict]))

In [ ]:
import os
val_filenames = ["/content/sentinel2imagery/S2A_MSIL2A_20200115T085311_N0213_R107_T36TVS_20200115T102431/T36TVS_20200115T085311_TCI_10m.jp2",
                 "/content/sentinel2imagery/S2A_MSIL2A_20190808T084601_N0213_R107_T36TVS_20190808T120658/T36TVS_20190808T084601_TCI_10m.jp2",
                 "/content/sentinel2imagery/S2A_MSIL2A_20190420T084601_N0211_R107_T36TVS_20190420T112440/T36TVS_20190420T084601_TCI_10m.jp2",
                 "/content/sentinel2imagery/S2A_MSIL2A_20191017T084941_N0213_R107_T36TVS_20191017T115536/T36TVS_20191017T084941_TCI_10m.jp2"]

test_filenames = ["/content/sentinel2imagery/S2B_MSIL2A_20211110T085109_N0301_R107_T36TVS_20211110T103058/T36TVS_20211110T085109_TCI_10m.jp2",
                  "/content/sentinel2imagery/S2B_MSIL2A_20200817T084559_N0214_R107_T36TVS_20200817T112210/T36TVS_20200817T084559_TCI_10m.jp2",
                  "/content/sentinel2imagery/S2B_MSIL2A_20200409T084559_N0214_R107_T36TVS_20200409T120319/T36TVS_20200409T084559_TCI_10m.jp2",
                  "/content/sentinel2imagery/S2B_MSIL2A_20190224T084939_N0211_R107_T36TVS_20190224T131744/T36TVS_20190224T084939_TCI_10m.jp2"]
crop_height = 480
crop_width = 640

dataset_dir = "/content/first_step_dataset"
os.makedirs(dataset_dir, exist_ok=True)

train_dir = os.path.join(dataset_dir, "train")
val_dir = os.path.join(dataset_dir, "val")
test_dir = os.path.join(dataset_dir, "test")

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

for dirname, _, filenames in os.walk('/content/sentinel2imagery'):
    for filename in filenames:
        if filename.endswith('_TCI_10m.jp2'):
          basename = os.path.basename(filename)[:-4]
          tilepath = os.path.join(dirname, filename)

          if tilepath in val_filenames:
            save_path = os.path.join(val_dir, basename)
          elif tilepath in test_filenames:
            save_path = os.path.join(test_dir, basename)
          else:
            save_path = os.path.join(train_dir, basename)


          image = load_torch_image(tilepath)

          split_image_and_save(
              image=image,
              crop_h=crop_height,
              crop_w=crop_width,
              save_crops=True,
              saving_path=save_path)




In [ ]:
import shutil
shutil.make_archive("/content/first_step_dataset", "zip", "/content/first_step_dataset/")